[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fireworks-ai/cookbook/blob/main/integrations/Klavis/Use_Klavis_with_Fireworks.ipynb)


# Fireworks AI + Klavis AI Integration

<div style="text-align: center">
<img src="../../assets/fw-klavis.png" width="400">
</div>

In this tutorial, we'll explore how to build an AI agent that integrates Fireworks AI's LLM with Klavis MCP Servers:


## Prerequisites

Before we begin, you'll need:

- **Fireworks AI API key** - see here [fireworks.ai](https://fireworks.ai/)
- **Klavis API key** - see here [klavis.ai](https://klavis.ai/)

Make sure to keep these API keys secure and never commit them to version control!


In [31]:
# Install the required packages
%pip install -qU fireworks-ai klavis


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import json
from fireworks.client import Fireworks
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["FIREWORKS_API_KEY"] = "fw_XXXXXXXXXXXXXX"  # Replace with your actual Fireworks API key
os.environ["KLAVIS_API_KEY"] = "XXXXXXXXXXXXX"    # Replace with your actual Klavis API key


## Create AI Agent with MCP Integration

Now we'll create an intelligent agent that can use MCP servers through Klavis API. This agent will:

1. **Discover Tools**: Automatically find available tools from MCP servers
2. **Function Calling**: Use Fireworks AI's function calling capabilities
3. **Tool Execution**: Execute tools through Klavis API
4. **Smart Responses**: Generate intelligent responses based on tool results


In [34]:
class Agent:
    def __init__(self, fireworks_client, klavis_client, mcp_server_url):
        self.fireworks = fireworks_client
        self.klavis = klavis_client
        self.mcp_server_url = mcp_server_url
        self.model = "accounts/fireworks/models/qwen2p5-72b-instruct"
        print(f"🤖 Agent initialized with model: {self.model}")
    
    def process_request(self, user_message):
        # 1. Get available tools
        mcp_tools = self.klavis.mcp_server.list_tools(
            server_url=self.mcp_server_url,
            connection_type=ConnectionType.STREAMABLE_HTTP,
            format=ToolFormat.OPENAI,
        )
        
        # 2. Call LLM with tools
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_message}
        ]
        
        response = self.fireworks.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=mcp_tools.tools
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # 3. If LLM wants to use tools
        if assistant_message.tool_calls:
            
            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)
                
                print(f"🛠️ Calling tool: {tool_name} with args: {tool_args}")
                # Call tool via Klavis SDK
                tool_result = self.klavis.mcp_server.call_tools(
                    server_url=self.mcp_server_url,
                    tool_name=tool_name,
                    tool_args=tool_args,
                    connection_type=ConnectionType.STREAMABLE_HTTP
                )
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(tool_result)
                })
            
            # 4. Get final response from LLM
            final_response = self.fireworks.chat.completions.create(
                model=self.model,
                messages=messages
            )
            return final_response.choices[0].message.content
        
        # If no tools needed, return the assistant message directly
        return assistant_message.content


## Use Case 1: Summarize YouTube Video

In [35]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=kPXvf2-C_Hs" # pick a video you like!

# 1. Initialize Fireworks AI client and Klavis client
fireworks_client = Fireworks(api_key=os.getenv("FIREWORKS_API_KEY"))
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

# 2. Create YouTube MCP server instance
youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# 3. Create an agent with YouTube MCP server
agent = Agent(fireworks_client, klavis_client, youtube_mcp_instance.server_url)

response = agent.process_request(
    f"Summarize this YouTube video with timestamps: {YOUTUBE_VIDEO_URL}"
)

print(response)

✅ Created YouTube MCP instance
🤖 Agent initialized with model: accounts/fireworks/models/qwen2p5-72b-instruct
🛠️ Calling tool: get_youtube_video_transcript with args: {'url': 'https://www.youtube.com/watch?v=kPXvf2-C_Hs'}
### Summary of the Video: "How This AI Startup Grew by 100x in Just 6 Months | Fireworks AI, Lin Qiao"

**Title:** How This AI Startup Grew by 100x in Just 6 Months | Fireworks AI, Lin Qiao  
**Published:** August 20, 2024  
**Channel:** EO  
**Duration:** 11 minutes 55 seconds  
**Views:** 77,278  
**Likes:** 1,650  
**Comments:** 51

#### **0:00 - 1:00: Introduction**
- Introduction to Fireworks AI, one of the fastest-growing AI startups in the industry.
- Over the past six months, Fireworks AI has seen a 100x increase in traffic.
- In July, the company closed a Series B funding round led by Sequoia, bringing its valuation to $552 million.

#### **1:00 - 3:30: Lin Qiao's Background**
- Lin Qiao, the founder of Fireworks AI, previously served as the Head of PyTorch.


## Use Case 2: Send Email

**Note**: Gmail integration requires OAuth authentication, so you'll need to authorize the application in your browser. 

In [ ]:
import webbrowser

# Create Gmail MCP server instance
gmail_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# Redirect to Gmail OAuth page
webbrowser.open(gmail_mcp_instance.oauth_url)
print(f"🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: {gmail_mcp_instance.oauth_url}")


In [30]:
EMAIL_SUBJECT = "Hello, World!"
EMAIL_BODY = "This is a test email."
EMAIL_RECIPIENT = "zihaolin@klavis.ai" # replace with your email

# Create an agent with Gmail MCP server
agent = Agent(fireworks_client, klavis_client, gmail_mcp_instance.server_url)

response = agent.process_request(
    f"Send an email to {EMAIL_RECIPIENT} with subject {EMAIL_SUBJECT} and body {EMAIL_BODY}"
)

print(response)

The email has been sent successfully to zihaolin@klavis.ai with the subject "Hello, World!" and the body "This is a test email." The email ID is 1977620de59daa96.


## 🎉 Conclusion

Congratulations! You've successfully built AI agents that can interact with any MCP server with Klavis. Here's what we accomplished:
 
### 🚀 Next Steps
- **Explore More MCP Servers**: Try other available servers like Slack, Notion, CRM, etc..
- **Try Different Fireworks AI Models**: Experiment with various models like Llama, Mixtral, or Deepseek for different use cases
- **Build Complex Multi-Server Workflows**: Create sophisticated agents that combine Gmail + Slack + Notion for complete business automation
- **Production Deployment**: Scale these patterns for production applications

### 🔗 Useful Resources
- [Fireworks AI Documentation](https://docs.fireworks.ai/)
- [Klavis AI Documentation](https://docs.klavis.ai/)
- [MCP Protocol Specification](https://modelcontextprotocol.io/)

**Happy building!** 🚀
